In [1]:
import tensorflow as tf

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
%load_ext autoreload
%autoreload 2



from pathlib import Path
import sys
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
import pickle
from sklearn.decomposition import PCA
from scipy import signal
from indl.display import turbo_cmap
from sklearn.model_selection import train_test_split
from indl.fileio import from_neuropype_h5
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

import os

if Path.cwd().stem == 'Analysis':
    os.chdir(Path.cwd().parent.parent)
    
    
data_path = Path.cwd() / 'StudyLocationRule'/ 'Data' / 'Preprocessed'
if not (data_path).is_dir():
    !kaggle datasets download --unzip --path {str(data_path)} cboulay/macaque-8a-spikes-rates-and-saccades
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")
    
from misc.misc import sess_infos, load_macaque_pfc, dec_from_enc

load_kwargs = {
    'valid_outcomes': (0,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

load_kwargs_error = {
    'valid_outcomes': (9,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

load_kwargs_all = {
    'valid_outcomes': (0,9),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'time_range': (-np.inf, 1.45),
    'verbose': False,
    'y_type': 'sacClass',
    'samples_last': True    
    #     'resample_X': 20
}

model_kwargs = dict(
    filt=8,
    kernLength=20,
    ds_rate=5,
    n_rnn=64,
    n_rnn2=0,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=64
)
model_kwargs1 = dict(
    filt=16,
    kernLength=30,
    ds_rate=5,
    n_rnn=64,
    n_rnn2=64,
    dropoutRate=0.40,
    activation='relu',
    l1_reg=0.0000, l2_reg=0.001,
    norm_rate=0.25,
    latent_dim=64
)

N_SPLITS = 10
BATCH_SIZE = 16
EPOCHS = 150
LABEL_SMOOTHING = 0.2

Data directory found. Skipping download.


In [4]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [5]:
test_sess_ix = 1
sess_info = sess_infos[test_sess_ix]
sess_id = sess_info['exp_code']
print(f"\nImporting session {sess_id}")
X_rates, Y, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', **load_kwargs)
Y_class = tf.keras.utils.to_categorical(Y, num_classes=8)


Importing session sra3_1_j_050_00+


In [6]:
number_of_classes = 8
n_timesteps, n_features = X_rates.shape[1], X_rates.shape[2]

print('input_shape', n_timesteps, n_features)
print('output', number_of_classes)

model = Sequential()
model.add(LSTM(100, batch_input_shape=(BATCH_SIZE,n_timesteps, n_features), return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(number_of_classes, activation='softmax'))

model.summary()

print("Created model.")

model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy', 
              metrics=['acc'])

print("Compiled model.")

input_shape 36 205
output 8
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (16, 36, 100)             122400    
_________________________________________________________________
lstm_1 (LSTM)                (16, 36, 100)             80400     
_________________________________________________________________
lstm_2 (LSTM)                (16, 100)                 80400     
_________________________________________________________________
dense (Dense)                (16, 8)                   808       
Total params: 284,008
Trainable params: 284,008
Non-trainable params: 0
_________________________________________________________________
Created model.
Compiled model.


In [7]:
trn = int(0.8*X_rates.shape[0])

_y = Y_class

ds_train = tf.data.Dataset.from_tensor_slices((X_rates[:trn], _y[:trn]))
ds_valid = tf.data.Dataset.from_tensor_slices((X_rates[trn:], _y[trn:]))

# cast data types to GPU-friendly types.
ds_train = ds_train.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))
ds_valid = ds_valid.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.uint8)))

# TODO: augmentations (random slicing?)

ds_train = ds_train.shuffle(trn + 1)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=True)

In [8]:
model.fit(x=ds_train, epochs=EPOCHS,validation_data=ds_valid, verbose=1)

Epoch 1/150
24/24 [==============================] - 6s 35ms/step - loss: 1.8628 - acc: 0.2895 - val_loss: 1.4408 - val_acc: 0.4688
Epoch 2/150
24/24 [==============================] - 0s 10ms/step - loss: 0.9652 - acc: 0.5938 - val_loss: 2.0771 - val_acc: 0.3542
Epoch 3/150
24/24 [==============================] - 0s 6ms/step - loss: 0.6256 - acc: 0.7714 - val_loss: 0.9120 - val_acc: 0.6667
Epoch 4/150
24/24 [==============================] - 0s 6ms/step - loss: 0.4441 - acc: 0.8420 - val_loss: 1.0547 - val_acc: 0.6458
Epoch 5/150
24/24 [==============================] - 0s 6ms/step - loss: 0.2551 - acc: 0.9222 - val_loss: 1.1037 - val_acc: 0.6146
Epoch 6/150
24/24 [==============================] - 0s 6ms/step - loss: 0.2661 - acc: 0.9046 - val_loss: 1.6680 - val_acc: 0.4792
Epoch 7/150
24/24 [==============================] - 0s 6ms/step - loss: 0.2369 - acc: 0.9330 - val_loss: 1.2726 - val_acc: 0.5833
Epoch 8/150
24/24 [==============================] - 0s 6ms/step - loss: 0.0911 -

24/24 [==============================] - 0s 6ms/step - loss: 3.7348e-04 - acc: 1.0000 - val_loss: 2.3074 - val_acc: 0.5938
Epoch 63/150
24/24 [==============================] - 0s 6ms/step - loss: 3.6739e-04 - acc: 1.0000 - val_loss: 2.3159 - val_acc: 0.6042
Epoch 64/150
24/24 [==============================] - 0s 6ms/step - loss: 3.5549e-04 - acc: 1.0000 - val_loss: 2.3209 - val_acc: 0.6042
Epoch 65/150
24/24 [==============================] - 0s 6ms/step - loss: 3.5583e-04 - acc: 1.0000 - val_loss: 2.3298 - val_acc: 0.5938
Epoch 66/150
24/24 [==============================] - 0s 6ms/step - loss: 3.5313e-04 - acc: 1.0000 - val_loss: 2.3394 - val_acc: 0.6042
Epoch 67/150
24/24 [==============================] - 0s 6ms/step - loss: 3.4117e-04 - acc: 1.0000 - val_loss: 2.3474 - val_acc: 0.5938
Epoch 68/150
24/24 [==============================] - 0s 6ms/step - loss: 3.4154e-04 - acc: 1.0000 - val_loss: 2.3548 - val_acc: 0.5938
Epoch 69/150
24/24 [==============================] - 0s 6ms/

24/24 [==============================] - 0s 6ms/step - loss: 1.0081e-04 - acc: 1.0000 - val_loss: 2.6693 - val_acc: 0.5625
Epoch 123/150
24/24 [==============================] - 0s 6ms/step - loss: 9.7562e-05 - acc: 1.0000 - val_loss: 2.6741 - val_acc: 0.5625
Epoch 124/150
24/24 [==============================] - 0s 6ms/step - loss: 9.5948e-05 - acc: 1.0000 - val_loss: 2.6790 - val_acc: 0.5625
Epoch 125/150
24/24 [==============================] - 0s 6ms/step - loss: 9.3781e-05 - acc: 1.0000 - val_loss: 2.6847 - val_acc: 0.5625
Epoch 126/150
24/24 [==============================] - 0s 6ms/step - loss: 9.4444e-05 - acc: 1.0000 - val_loss: 2.6898 - val_acc: 0.5625
Epoch 127/150
24/24 [==============================] - 0s 6ms/step - loss: 9.3683e-05 - acc: 1.0000 - val_loss: 2.6951 - val_acc: 0.5625
Epoch 128/150
24/24 [==============================] - 0s 6ms/step - loss: 8.9433e-05 - acc: 1.0000 - val_loss: 2.6989 - val_acc: 0.5625
Epoch 129/150
24/24 [==============================] - 

# MNIST

In [9]:
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

In [7]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /home/alireza/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [8]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)


In [9]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


In [10]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)


Epoch 1/6
469/469 [==============================] - 1s 1ms/step - loss: 0.3562 - sparse_categorical_accuracy: 0.9033 - val_loss: 0.1957 - val_sparse_categorical_accuracy: 0.9428
Epoch 2/6
469/469 [==============================] - 0s 1ms/step - loss: 0.1609 - sparse_categorical_accuracy: 0.9544 - val_loss: 0.1361 - val_sparse_categorical_accuracy: 0.9609
Epoch 3/6
469/469 [==============================] - 0s 987us/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9680 - val_loss: 0.1100 - val_sparse_categorical_accuracy: 0.9668
Epoch 4/6
469/469 [==============================] - 0s 1ms/step - loss: 0.0895 - sparse_categorical_accuracy: 0.9745 - val_loss: 0.1009 - val_sparse_categorical_accuracy: 0.9698
Epoch 5/6
469/469 [==============================] - 1s 1ms/step - loss: 0.0711 - sparse_categorical_accuracy: 0.9798 - val_loss: 0.0836 - val_sparse_categorical_accuracy: 0.9738
Epoch 6/6
469/469 [==============================] - 0s 1ms/step - loss: 0.0603 - sparse_categorical_ac